In [34]:
# Importing Library and setting environment path
import os
import sys
# set the path 

sparkPath = "C:/Users/Pranathi/Downloads/spark-2.0.2-bin-hadoop2.7/spark-2.0.2-bin-hadoop2.7"

os.environ['SPARK_HOME'] = sparkPath
sys.path.append(sparkPath + '/bin')
sys.path.append(sparkPath + '/python')
sys.path.append(sparkPath + '/python/pyspark')
sys.path.append(sparkPath + '/python/pyspark/lib')
sys.path.append(sparkPath + '/python/pyspark/lib/pyspark.zip')
sys.path.append(sparkPath + '/python/pyspark/lib/py4j-0.10.3-src.zip')
sys.path.append("C:/Program Files (x86)/Java/jre1.8.0_111/bin")

import pyspark
from pyspark.streaming import StreamingContext

#ssc.stop()
sc = pyspark.SparkContext("local[2]", "trafficData")
ssc = StreamingContext(sc, 10)  # Seconds per micro-batch
sqlContext = pyspark.SQLContext(sc)

streamPath = 'http://207.251.86.229/nyc-links-cams/LinkSpeedQuery.txt'
streamFilePath = "file:/C:/Users/Pranathi/Desktop/MSBA/Fall/Harvesting Big Data/Project/data/"

import datetime
today = datetime.datetime.today()
currentDay = today.day
currentMonth = today.month
currentYear = today.year

import pandas as pd

In [35]:
trafficStream = ssc.textFileStream(streamFilePath).map(lambda line: line.split(','))

trafficStream.map(lambda line: (line[0], line[1], line[2], line[4],line[11], line[12])).foreachRDD(lambda rdd: rdd.toDF().registerTempTable("traffic"))
#trafficStream.pprint()
ssc.start()

In [36]:
#Setting the layout for average speed
import numpy as np 
import plotly.plotly as py  
import plotly.tools as tls   
import plotly.graph_objs as go
import plotly 
from scipy import stats

plotly.tools.set_credentials_file(username='pranathim1', api_key='PkS3DJye5w3qpFPjzBN1')

token_2 = 'xoyxw205yt'
stream_id2 = dict(token=token_2, maxpoints=5)

trace1 = go.Bar(x=[], y=[], stream=stream_id2, name='Average Speed',marker =
                dict(color = ('rgb(120, 232, 187)')))

data = [trace1]

layout = dict(title = 'Average Speeds in Boroughs of New York',
              xaxis = dict(title = 'Borough'),
              yaxis = dict(title = 'Average Speed (miles/hour)'),
              )

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='grouped-bar')


In [38]:
s_2 = py.Stream(stream_id=token_2)
s_2.open()


import time
import datetime
import numpy as np

while True:
    try:
        trafficDF = sqlContext.sql("select * from traffic LIMIT 87").toPandas()  # 155 instances updated each time
        trafficDF.columns = ['ID', 'Speed', 'TravelTime', 'Time', 'Borough', 'LinkName']  # Naming the columns
    
    # Removing records not in current year/month/day
        trafficDF['Time'] = pd.to_datetime(trafficDF['Time'])  # Converting time to datetime
        trafficDF = trafficDF[trafficDF['Time'].dt.year == currentYear]
        trafficDF = trafficDF[trafficDF['Time'].dt.month == currentMonth]
        trafficDF = trafficDF[trafficDF['Time'].dt.day == currentDay]
   
        trafficDF['Speed'] = trafficDF['Speed'].astype(float)
        trafficDF['TravelTime'] = trafficDF['TravelTime'].astype(float)
    
        averageDF = trafficDF.groupby('Borough').mean().reset_index(drop=False)
    
        x = averageDF['Borough']
        y = averageDF['Speed']
    #y2 = averageDF['TravekTime']
        s_2.write(dict(x=x, y=y))
        time.sleep( 3 )
    except KeyboardInterrupt:
        break
        
s_2.close()

In [24]:
trafficDF = sqlContext.sql("select * from traffic LIMIT 87").toPandas()

In [33]:
ssc.stop()